# WEB SCRAPING JOB PROFILES

## ABSTRACT

The objective in this script is to extract Job profiles of Data Analysts in Massachusetts from https://Indeed.com. The prime source of extraction is through web scarping and the data is munged to obtain a consistent dataset. The data is extracted using Selenium and BeautifulSoup. Furthermore, the dataset is cleaned and audited to make it unique and error free.

The conceptual form of the database is shown in the ER diagram.

Once the cleaned dataset is obtained, the dataframe is converted to SQL using SQLAlchemy and the data is inserted. The database engine sqlite is then connected to jupyter notebook. Any queries can then be executed from jupyter notebook and the output is displayed.

#### Installing pip packages selenuim and ipyhton

In [1]:
!pip install selenium
!pip install ipython

#### Importing libraries for webdriver, pandas, webscraping and SQLalchemy

In [25]:
from selenium import webdriver
import pandas as pd 
import pprint
import os
from sqlalchemy import create_engine
from bs4 import BeautifulSoup

In [26]:
job_titles=[]
companies=[]
locations=[]
summaries=[]
salaries=[]
ratings=[]

#### Specifying path to chromedriver.exe

In [37]:
driver = webdriver.Chrome('/Users/georg/chromedriver.exe')

## WEB SCRAPING

#### Scraping first 40 pages from https://www.Indeed.com

In [38]:
for i in range(0,400,10):
    driver.get('https://www.indeed.com/jobs?q=Data+Analyst&l=Massachusetts&start='+str(i))
    jobs = []
    driver.implicitly_wait(4)
    
    for job in driver.find_elements_by_class_name('result'):

        soup = BeautifulSoup(job.get_attribute('innerHTML'),'html.parser')
        
        try:
            job_title = soup.find("a",class_="jobtitle").text.replace("\n","").strip()
            
        except:
            title = 'None'
        job_titles.append(job_title)

        try:
            location = soup.find(class_="location").text
        except:
            location = 'None'
        locations.append(location)

        try:
            company = soup.find(class_="company").text.replace("\n","").strip()
        except:
            company = 'None'
        companies.append(company)

        try:
            salary = soup.find(class_="salary").text.replace("\n","").strip()
        except:
            salary = 'None'
        salaries.append(salary)
        
        try:
            summary = soup.find('div', class_= 'summary').text.strip()
        except:
            summary = 'None'
        summaries.append(summary)
        
        try:
            rating = soup.find('span', class_= 'ratingsContent').text.strip()
        except:
            rating = 'None'
        ratings.append(rating)

        
        pop_up = job.find_element_by_xpath('./div[3]')
        try:
            pop_up.click()
        except:
            close_button = driver.find_elements_by_class_name('popover-x-button-close')[0]
            close_button.click()
            pop_up.click()

#### Displaying the scraped data

In [39]:
for t in job_titles:
    if t == None:
        t = ""
    print("Job Title: " + t)

for c in companies:
    if c == None:
        c = ""
    print("Company: " + c)

for l in locations:
    if l == None:
        l = ""
    print("Location: " + l)

for su in summaries:
    if su == None:
        su = ""
    print("Summary: " + su)

for s in salaries:
    if s == None:
        s = ""
    print("Salary: " + s)

for r in ratings:
    if r == None:
        r = ""
    print("Rating: " + r)

Job Title: Analyst II, Data Science
Job Title: Data Analytics/Digital Marketing Analyst
Job Title: Business Analyst, Global Scientific Workflows
Job Title: Business Data/Business Intelligence Analyst
Job Title: Data Analyst
Job Title: Entry Level Data Analyst
Job Title: Data Analyst / Quantitative Data Analyst
Job Title: Data Analyst
Job Title: Data Analyst Health Care
Job Title: Data Analyst
Job Title: Data Analyst
Job Title: Lab Data Analyst - Life Sciences
Job Title: Data Analyst
Job Title: RESEARCH DATA ANALYST, School of Medicine, Slone Epidemiolog...
Job Title: Sales Analyst
Job Title: Data Analyst
Job Title: System Analyst/Data Analyst - MDM required
Job Title: Healthcare Data Analyst
Job Title: Sales Analyst
Job Title: Data Analytics Intern
Job Title: Analytics Business Analyst
Job Title: Data Analyst
Job Title: Data Analyst
Job Title: Product Data Analyst
Job Title: Data Analyst
Job Title: Data Analyst
Job Title: Data Analyst
Job Title: Data Analyst
Job Title: Data Analyst for

#### JSON consolidation

In [40]:
jobs_df = []

for i in range(0,len(job_titles)-1):
    jobs_json = {
        "Job_Title":job_titles[i],
        "Company":companies[i],
        "Location":locations[i],
        "Summary":summaries[i],
        "Salary":salaries[i],
        "Rating":ratings[i]
    }
    jobs_df.append(jobs_json)

## CLEANING AND AUDITING

In [41]:
jobs_data = pd.DataFrame(jobs_df, columns = ["Job_Title","Company","Location","Summary","Salary","Rating"])

In [42]:
jobs_data.describe()

,Job_Title,Company,Location,Summary,Salary,Rating
count,1316,1316,1316,1316,1316,1316
unique,238,234,127,281,23,23
top,Data Analyst,Boston Children's Hospital,"Boston, MA",As we begin evolving our data delivery model f...,None,None
freq,261,43,220,41,1083,343


#### Finding the duplicates rows

In [43]:
duplicate_rows = jobs_data[jobs_data.duplicated()]
print(duplicate_rows)

                                              Job_Title  \
21                                         Data Analyst   
32    RESEARCH DATA ANALYST, School of Medicine, Slo...   
42                                         Data Analyst   
45                                         Data Analyst   
56                             Data Analyst, Operations   
...                                                 ...   
1307                   Response Center Analyst - Tier 1   
1312                       Risk Adjustment Data Analyst   
1313                                      Sales Analyst   
1314                     Data Analytics Project Manager   
1315   Associate Director, Forecasting & Data Analytics   

                               Company  \
21                            Clinedge   
32                   Boston University   
42                        Congruity360   
45                      Jerneltechcorp   
56                        The CM Group   
...                                ... 

#### Removing the duplicates

In [44]:
clean_jobs_data = jobs_data.drop_duplicates(keep="first")
clean_jobs_data.describe()

,Job_Title,Company,Location,Summary,Salary,Rating
count,289,289,289,289,289,289
unique,238,234,127,281,23,23
top,Business Analyst,State Street,"Boston, MA","We work with MySQL, Python, R, Excel, and more...",None,None
freq,21,4,48,2,262,71


#### Checking for uniqueness

In [45]:
test = clean_jobs_data.copy()
test['ID'] = test['Company'] + test['Job_Title']
test['ID'].is_unique

False

#### Creating and appending Job ID

In [46]:
clean_jobs_data['Job_ID'] = range(1, 1+len(clean_jobs_data))
clean_jobs_data

C:\Users\georg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Job_Title,Company,Location,Summary,Salary,Rating,Job_ID
0,"Analyst II, Data Science",Liberty Mutual Insurance,"Boston, MA 02101",Collaborate with business partners to develop ...,"$89,700 - $148,800 a year",3.6,1
1,Data Analytics/Digital Marketing Analyst,Exceptional Lives https://exceptionallives.org,"Boston, MA 02109 (Downtown area)",5 years of experience of supporting digital or...,"$70,000 - $90,000 a year",None,2
2,"Business Analyst, Global Scientific Workflows",Takeda Pharmaceuticals,"Cambridge, MA 02139 (Cambridgeport area)",5-8 years of experience preferably in the phar...,None,3.8,3
3,Business Data/Business Intelligence Analyst,Acara Solutions,"Foxborough, MA",Minimum 5 years of business experience as a bu...,None,3.4,4
4,Data Analyst,Tufts University,"Boston, MA 02111 (Chinatown area)",1-3 years experience required.\nThe Nutrition ...,"$50,000 - $60,000 a year",4.1,5
...,...,...,...,...,...,...,...
1305,Business Analyst,Mass School Building Authority,"Boston, MA",Work with stakeholders and end users to design...,"$72,999 - $87,599 a year",3.9,285
1308,Workday Security and Business Process Analyst,PerkinElmer,"Waltham, MA 02451","Must be a detail oriented, self-starter with e...",None,3.7,286
1309,Sr. Business Analyst,UST Global,"Boston, MA","Participates in an agile development team, ens...",None,3.7,287
1310,Data Steward and Management Analyst,CRICO,"Boston, MA 02215 (Kenmore area)",Minimum 3 years’ experience as a data steward ...,None,None,288


#### Converting to csv file 

In [16]:
clean_jobs_data.to_csv("job_file.csv",encoding="utf-8",index=False)

## BUILDING DB SCHEMA AND INSERTING DATA

#### Converting dataframes to sql

In [48]:
engine = create_engine('sqlite://', echo=False)

#### SQL queries output

In [66]:
clean_jobs_data.to_sql('user', if_exists='replace',con=engine, index=False)
job_list = engine.execute("SELECT * FROM user").fetchall()


for i in job_list:
    print("Job ID: {0}\nJob Title: {1}\nCompany: {2}\nLocation: {3}\nSummary: {4}\nSalary: {5}\nRating: {6}\n".format(i[6], i[0], i[1], i[2], i[3], i[4], i[5]))

Job ID: 1
Job Title: Analyst II, Data Science
Company: Liberty Mutual Insurance
Location: Boston, MA 02101
Summary: Collaborate with business partners to develop predictive analytic solutions that enable data-driven strategic decision-making.
Salary: $89,700 - $148,800 a year
Rating: 3.6

Job ID: 2
Job Title: Data Analytics/Digital Marketing Analyst
Company: Exceptional Lives https://exceptionallives.org
Location: Boston, MA 02109 (Downtown area)
Summary: 5 years of experience of supporting digital organizations with all analytics needs,.
5 years of experience with Google Analytics and Google Tag Manager.
Salary: $70,000 - $90,000 a year
Rating: None

Job ID: 3
Job Title: Business Analyst, Global Scientific Workflows
Company: Takeda Pharmaceuticals
Location: Cambridge, MA 02139 (Cambridgeport area)
Summary: 5-8 years of experience preferably in the pharmaceutical or life sciences industry.
As part of Scientific Informatics, the Global Scientific Workflows team has…
Salary: None
Rating:

## USE CASES

Queries can be written to find jobs with certain keywords in the job description. An example below shows all jobs with keyword "Python" in their job description.

In [67]:
clean_jobs_data.to_sql('user', if_exists='replace',con=engine, index=False)
job_list = engine.execute("SELECT * FROM user WHERE Summary LIKE '%Python%'").fetchall()


for i in job_list:
    print("Job ID: {0}\nJob Title: {1}\nCompany: {2}\nLocation: {3}\nSummary: {4}\nSalary: {5}\nRating: {6}\n".format(i[6], i[0], i[1], i[2], i[3], i[4], i[5]))

Job ID: 6
Job Title: Entry Level Data Analyst
Company: Hopjump
Location: Boston, MA
Summary: We work with MySQL, Python, R, Excel, and more, though no prior knowledge of programming is required.
We're looking for analytical thinkers interested in…
Salary: None
Rating: None

Job ID: 7
Job Title: Data Analyst / Quantitative Data Analyst
Company: Hopjump
Location: Cambridge, MA
Summary: We work with MySQL, Python, R, Excel, and more, though no prior knowledge of programming is required.
We're looking for analytical thinkers interested in…
Salary: None
Rating: None

Job ID: 99
Job Title: Data Engineering Analyst
Company: Dun & Bradstreet
Location: Waltham, MA 02451
Summary: Experience with SQL (1-3 years).
Experience in application development [python preferred] (1-3 years).
The Data Engineering Analyst will have to both be able to…
Salary: None
Rating: 3.7

Job ID: 127
Job Title: Senior Data Analyst
Company: John Hancock
Location: Boston, MA
Summary: Extract, consolidate, validate and tra

These queries can also be used to find the the preffered location by users. For example if a user is looking for jobs only
around cambridge, the following query can be used

In [69]:
clean_jobs_data.to_sql('user', if_exists='replace',con=engine, index=False)
job_list = engine.execute("SELECT * FROM user WHERE Location LIKE '%Cambridge%'").fetchall()


for i in job_list:
    print("Job ID: {0}\nJob Title: {1}\nCompany: {2}\nLocation: {3}\nSummary: {4}\nSalary: {5}\nRating: {6}\n".format(i[6], i[0], i[1], i[2], i[3], i[4], i[5]))

Job ID: 3
Job Title: Business Analyst, Global Scientific Workflows
Company: Takeda Pharmaceuticals
Location: Cambridge, MA 02139 (Cambridgeport area)
Summary: 5-8 years of experience preferably in the pharmaceutical or life sciences industry.
As part of Scientific Informatics, the Global Scientific Workflows team has…
Salary: None
Rating: 3.8

Job ID: 7
Job Title: Data Analyst / Quantitative Data Analyst
Company: Hopjump
Location: Cambridge, MA
Summary: We work with MySQL, Python, R, Excel, and more, though no prior knowledge of programming is required.
We're looking for analytical thinkers interested in…
Salary: None
Rating: None

Job ID: 13
Job Title: Data Analyst
Company: CareDash
Location: Cambridge, MA
Summary: Data is the foundation of our business and drives all efforts at CareDash including supporting cross-functional projects across marketing operations, healthcare…
Salary: None
Rating: 5.0

Job ID: 15
Job Title: Sales Analyst
Company: Philips
Location: Cambridge, MA
Summary: 

If a user is looking for jobs with rating 4.0 and above only, the code below can be used. 

In [72]:
clean_jobs_data.to_sql('user', if_exists='replace',con=engine, index=False)
job_list = engine.execute("SELECT * FROM user WHERE Rating>4.0 AND Rating NOT LIKE'None'").fetchall()


for i in job_list:
    print("Job ID: {0}\nJob Title: {1}\nCompany: {2}\nLocation: {3}\nSummary: {4}\nSalary: {5}\nRating: {6}\n".format(i[6], i[0], i[1], i[2], i[3], i[4], i[5]))

Job ID: 5
Job Title: Data Analyst
Company: Tufts University
Location: Boston, MA 02111 (Chinatown area)
Summary: 1-3 years experience required.
The Nutrition Innovation Lab is seeking a full-time* Data Analyst position that will be based in Boston at the Friedman School.
Salary: $50,000 - $60,000 a year
Rating: 4.1

Job ID: 9
Job Title: Data Analyst Health Care
Company: Tufts University
Location: Boston, MA 02111 (Chinatown area)
Summary: 1-3 years experience required.
The Nutrition Innovation Lab is seeking a full-time* Data Analyst position that will be based in Boston at the Friedman School.
Salary: $50,000 - $60,000 a year
Rating: 4.1

Job ID: 13
Job Title: Data Analyst
Company: CareDash
Location: Cambridge, MA
Summary: Data is the foundation of our business and drives all efforts at CareDash including supporting cross-functional projects across marketing operations, healthcare…
Salary: None
Rating: 5.0

Job ID: 14
Job Title: RESEARCH DATA ANALYST, School of Medicine, Slone Epidemi

### In the script 'Company Twitter Tags', we have continued the project. We have gone on to get Company Tweets to draw insightful conclusions.  